In [1]:
from killer_tick.tick import KillTick
import numpy as np
import pandas as pd
import datetime as dt
import holidays
from pytz import timezone

tz = timezone('America/Mexico_City')

In [2]:
yesterday = dt.datetime.now(tz) - dt.timedelta(days=1)
last_60_day = yesterday - dt.timedelta(days=60)

In [3]:
client = KillTick()

In [4]:
# 1. API call
raw_clients = client.get_clients()
raw_projects = client.get_projects()
raw_tasks = client.get_tasks()
raw_entries = client.get_entries(start_date=last_60_day.strftime("%Y-%m-%d"),
                                 end_date=yesterday.strftime("%Y-%m-%d"))

# 2. Seleccionar las columnas necesarias
clients = raw_clients.loc[:, ['id', 'name']]
projects = raw_projects.loc[:, ['id', 'name', 'client_id']]
tasks = raw_tasks.loc[:, ['id', 'name', 'project_id']]
entries = raw_entries.loc[:, ['id', 'date', 'hours', 'notes', 'task_id']]

# 3. Unir Clientes, proyectos, tareas y entradas por ID
m_tasks = entries.merge(tasks, left_on='task_id', right_on='id', how='left')
m_tasks.drop(['id_x', 'id_y'], axis=1, inplace=True)
m_projects = m_tasks.merge(projects, left_on='project_id',
                           right_on='id', how='left', suffixes=["_task", "_project"])
m_projects.drop("id", axis=1, inplace=True)
c_clients = m_projects.merge(
    clients, left_on='client_id', right_on='id', how='left')
c_clients.drop("id", axis=1, inplace=True)
tick_records = c_clients.rename(columns={'name_task': 'task_name',
                                         'name_project': 'project_name',
                                         'name': 'client_name'})

# 4. Calcular el total de horas el el periodo
tick_records.hours.sum()

344.0

In [5]:
tick_records.sort_values("date", ascending=False)

,date,hours,notes,task_id,task_name,project_id,project_name,client_id,client_name
32,2022-08-05,8.0,,16817814,PROYECTOS IT EPA,2090949,EPA DIGITAL,412134,PROYECTO EPA
33,2022-08-04,8.0,,16817814,PROYECTOS IT EPA,2090949,EPA DIGITAL,412134,PROYECTO EPA
34,2022-08-03,8.0,,16817814,PROYECTOS IT EPA,2090949,EPA DIGITAL,412134,PROYECTO EPA
35,2022-08-02,8.0,,16817814,PROYECTOS IT EPA,2090949,EPA DIGITAL,412134,PROYECTO EPA
36,2022-08-01,8.0,test API call,16817814,PROYECTOS IT EPA,2090949,EPA DIGITAL,412134,PROYECTO EPA
...,...,...,...,...,...,...,...,...,...
17,2022-06-09,2.0,,15826705,Atencion al cliente,1967121,AMVO,411327,AMVO
18,2022-06-09,2.0,,15830419,Atencion al cliente,2083099,Innovasport,322387,Innovasport
11,2022-06-08,2.0,,15826702,Reportes,1967121,AMVO,411327,AMVO
20,2022-06-08,1.0,,15914025,STATUS EQUIPO (no cliente),2090949,EPA DIGITAL,412134,PROYECTO EPA


In [6]:
last_entry_date = (dt.datetime.strptime(tick_records.date.max(), '%Y-%m-%d') + dt.timedelta(days=1)).replace(tzinfo=tz)
mx_holidays = holidays.country_holidays('MX', years=dt.datetime.today().year)
days_off = [i[0] for i in mx_holidays.items()]
remaining_days = pd.bdate_range(last_entry_date, yesterday,
                                freq='C',
                                holidays=days_off).date


In [ ]:
[client.create_entry({"date": str(i),
        "hours": 8,
        "notes": "",
        "task_id": 16817814}) for i in remaining_days]